In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

%reload_ext tensorboard

time: 13.1 ms (started: 2021-09-02 13:29:32 +08:00)


In [2]:
%%bash

# 增加更新
git add *.ipynb */*.ipynb

git remote -v

git commit -m '更新 #3 Sept 02, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/pytorch_lightning_study.git (fetch)
origin	git@github.com:ustchope/pytorch_lightning_study.git (push)
[main e4b8052] 更新 #3 Sept 02, 2021
 2 files changed, 12 insertions(+)
 create mode 100644 "Lightning-Bolts/.ipynb_checkpoints/\346\234\252\345\221\275\345\220\215-checkpoint.ipynb"
 create mode 100644 "Lightning-Bolts/\346\234\252\345\221\275\345\220\215.ipynb"


To github.com:ustchope/pytorch_lightning_study.git
   5e5af48..e4b8052  main -> main


time: 3.84 s (started: 2021-09-02 09:24:18 +08:00)


In [3]:
import os

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from collections import OrderedDict

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything, Trainer, LightningDataModule
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torch.optim.swa_utils import AveragedModel, update_bn
from torchmetrics.functional import accuracy
import torchmetrics
import torchvision.transforms as transforms

seed_everything(7)

PATH_DATASETS = os.environ.get('PATH_DATASETS', '.')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count() / 2)

Global seed set to 7


time: 90.7 ms (started: 2021-09-02 13:29:34 +08:00)


**介绍指南**

欢迎使用 PyTorch Lightning Bolts！

Bolts 是一个深度学习研究和生产工具箱：
* SOTA 预训练模型。
* 模型组件。
* 回调。
* 损失。
* 数据集。

**Bolts 的主要目标是尽可能快地尝试新想法！**

所有模型都经过（每天）测试、基准测试、记录并在 CPU、TPU、GPU 和 16 位精度上工作。

一些例子！

In [4]:
from pl_bolts.models import VAE
from pl_bolts.models.vision import GPT2, ImageGPT, PixelCNN
from pl_bolts.models.self_supervised import AMDIM, CPC_v2, SimCLR, Moco_v2
from pl_bolts.models import LinearRegression, LogisticRegression
from pl_bolts.models.gans import GAN
from pl_bolts.callbacks import PrintTableMetricsCallback
from pl_bolts.datamodules import FashionMNISTDataModule, CIFAR10DataModule, ImagenetDataModule

time: 20.6 ms (started: 2021-09-02 09:26:24 +08:00)


**Bolts 是为快速的想法迭代而构建的 - 子类化、覆盖和训练！**

In [ ]:
from pl_bolts.models.vision import ImageGPT
from pl_bolts.models.self_supervised import SimCLR

class VideoGPT(ImageGPT):

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = _shape_input(x)

        logits = self.gpt(x)
        simclr_features = self.simclr(x)

        # -----------------
        # do something new with GPT logits + simclr_features
        # -----------------

        loss = self.criterion(logits.view(-1, logits.size(-1)), x.view(-1).long())

        logs = {"loss": loss}
        return {"loss": loss, "log": logs}

随意混合和匹配数据、模块和组件！

In [ ]:
model = GAN(datamodule=ImagenetDataModule(PATH))
model = GAN(datamodule=FashionMNISTDataModule(PATH))
model = ImageGPT(datamodule=FashionMNISTDataModule(PATH))

并在任何硬件加速器上训练

In [ ]:
import pytorch_lightning as pl

model = ImageGPT(datamodule=FashionMNISTDataModule(PATH))

# cpus
pl.Trainer.fit(model)

# gpus
pl.Trainer(gpus=8).fit(model)

# tpus
pl.Trainer(tpu_cores=8).fit(model)model = ImageGPT()
Trainer().fit(
    model,
    train_dataloader=DataLoader(...),
    val_dataloader=DataLoader(...)
)

或传入您选择的任何数据集

In [ ]:
model = ImageGPT()

Trainer().fit(
    model,
    train_dataloader=DataLoader(...),
    val_dataloader=DataLoader(...)
)

## 社区建设

然后闪电社区构建螺栓并将它们贡献给螺栓。 闪电团队保证贡献是：
* 经过严格测试（CPU、GPU、TPU）。
* 有据可查。
* 通过 PyTorch 闪电标准化。
* 针对速度进行了优化。
* 检查正确性。

# 何时使用Blots

## 对于预训练模型

大多数螺栓都有在各种数据集或算法上训练的预训练权重。 当您没有足够的数据、时间或金钱来进行自己的培训时，这很有用。

例如，您可以使用预训练的 VAE 为图像数据集生成特征。

In [ ]:
from pl_bolts.models.autoencoders import VAE
from pl_bolts.models.self_supervised import CPC_v2

model1 = VAE(input_height=32, pretrained='imagenet2012')
encoder = model1.encoder
encoder.eval()

# bolts are pretrained on different datasets
model2 = CPC_v2(encoder='resnet18', pretrained='imagenet128').freeze()
model3 = CPC_v2(encoder='resnet18', pretrained='stl10').freeze()

In [ ]:
for (x, y) in own_data:
    features = encoder(x)
    feat2 = model2(x)
    feat3 = model3(x)

# which is better?

## 微调您的数据

如果您有自己的数据，微调通常可以提高性能。 由于这是纯 PyTorch，您可以使用您喜欢的任何微调协议。

示例 1：解冻的微调

In [ ]:
# unfrozen finetune
model = CPC_v2(encoder='resnet18', pretrained='imagenet128')
resnet18 = model.encoder
# don't call .freeze()

classifier = LogisticRegression(...)

for (x, y) in own_data:
    feats = resnet18(x)
    y_hat = classifier(feats)

示例 2：先冻结再解冻

In [ ]:
# FREEZE!
model = CPC_v2(encoder='resnet18', pretrained='imagenet128')
resnet18 = model.encoder
resnet18.eval()

classifier = LogisticRegression(...)

for epoch in epochs:
    for (x, y) in own_data:
        feats = resnet18(x)
        y_hat = classifier(feats)
        loss = cross_entropy_with_logits(y_hat, y)

    # UNFREEZE after 10 epochs
    if epoch == 10:
        resnet18.unfreeze()

## 研究用

这是 bolts 与其他带有模型的库非常不同的地方。 它不仅是为生产而设计的，而且每个模块都经过编写，可以轻松扩展以进行研究。

In [ ]:
from pl_bolts.models.vision import ImageGPT
from pl_bolts.models.self_supervised import SimCLR

class VideoGPT(ImageGPT):

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = _shape_input(x)

        logits = self.gpt(x)
        simclr_features = self.simclr(x)

        # -----------------
        # do something new with GPT logits + simclr_features
        # -----------------

        loss = self.criterion(logits.view(-1, logits.size(-1)), x.view(-1).long())

        logs = {"loss": loss}
        return {"loss": loss, "log": logs}

或者，您的研究是在 self_supervised_learning 中，并且您想要做一个新的 SimCLR。 在这种情况下，您唯一想要改变的就是损失。

通过子类化，您可以专注于更改系统的单个部分，而不必担心其他部分会起作用（因为如果它们在 Bolt 中，那么它们会起作用并且我们已经对其进行了测试）。

In [ ]:
# subclass SimCLR and change ONLY what you want to try
class ComplexCLR(SimCLR):

    def init_loss(self):
        return self.new_xent_loss

    def new_xent_loss(self):
        out = torch.cat([out_1, out_2], dim=0) n_samples = len(out)

        # Full similarity matrix
        cov = torch.mm(out, out.t().contiguous())
        sim = torch.exp(cov / temperature)

        # Negative similarity
        mask = ~torch.eye(n_samples, device=sim.device).bool()
        neg = sim.masked_select(mask).view(n_samples, -1).sum(dim=-1)

        # ------------------
        # some new thing we want to do
        # ------------------

        # Positive similarity :
        pos = torch.exp(torch.sum(out_1 * out_2, dim=-1) / temperature)
        pos = torch.cat([pos, pos], dim=0)
        loss = -torch.log(pos / neg).mean()

        return loss

# 回调

回调是任意程序，可以在 Lightning 的训练循环中的任何时间点运行。

Bolts 包含一组社区贡献的回调，可以在任何 Lightning 模块中工作！

In [ ]:
from pl_bolts.callbacks import PrintTableMetricsCallback
import pytorch_lightning as pl

trainer = pl.Trainer(callbacks=[PrintTableMetricsCallback()])

## 信息回调

这些回调在训练期间提供了各种有用的信息。

### 打印表指标

此回调将训练指标打印到表格中。 出于速度目的，它非常简单。

In [ ]:
from pl_bolts.callbacks import PrintTableMetricsCallback

callback = PrintTableMetricsCallback()

trainer = pl.Trainer(callbacks=[callback])
trainer.fit(...)

# ------------------------------
# at the end of every epoch it will print
# ------------------------------

# loss│train_loss│val_loss│epoch
# ──────────────────────────────
# 2.2541470527648926│2.2541470527648926│2.2158432006835938│0

### LightningModule 中的数据监控

数据监控回调允许您记录和检查通过训练步骤和模型层的数据分布。 当与支持的记录器结合使用时，TrainingDataMonitor 为 training_step() 中的每个批次输入创建直方图并将其发送到记录器：

In [ ]:
from pl_bolts.callbacks import TrainingDataMonitor
from pytorch_lightning import Trainer

# 记录发送到 LightningModule.training_step 的输入数据的直方图
monitor = TrainingDataMonitor(log_every_n_steps=25)

model = YourLightningModule()
trainer = Trainer(callbacks=[monitor])
trainer.fit()

第二个更高级的 ModuleDataMonitor 回调跟踪通过模型本身及其子模块的数据的直方图，即它跟踪所有 .forward() 调用并注册输入和输出。 您可以跟踪所有或仅跟踪一部分子模块：

In [ ]:
from pl_bolts.callbacks import ModuleDataMonitor
from pytorch_lightning import Trainer

# log the in- and output histograms of LightningModule's `forward`
monitor = ModuleDataMonitor()

# LightningModule 中的所有子模块
monitor = ModuleDataMonitor(submodules=True)

# specific submodules
monitor = ModuleDataMonitor(submodules=["generator", "generator.conv1"])

model = YourLightningModule()
trainer = Trainer(callbacks=[monitor])
trainer.fit()

这对于调试复杂模型中的数据流和识别数值不稳定性特别有用。

### 模型验证

批量优化的梯度检查

一批样本的梯度下降不仅有利于优化，还可以利用数据并行性。 但是，必须注意不要在批次维度上混合数据。 只有在重塑或排列操作中的一个小错误会导致优化卡住，你甚至不会得到运行时错误。 如何判断模型是否在批次中混合了数据？ 一个简单的技巧是执行以下操作：
* 在示例批次上运行模型（可以是随机数据）
* 获取输出批次并选择第 n 个样本（选择 n）
* 仅计算该样本的虚拟损失值并计算整个输入批次的梯度
* 观察到只有输入批次中的第 i 个样本具有非零梯度

如果批次中其他样本的梯度不为零，则意味着模型的前向传递正在混合数据！ BatchGradientVerificationCallback 在训练开始之前为您完成所有这些工作。

In [ ]:
from pytorch_lightning import Trainer
from pl_bolts.callbacks import BatchGradientVerificationCallback

model = YourLightningModule()
verification = BatchGradientVerificationCallback()
trainer = Trainer(callbacks=[verification])
trainer.fit(model)

如果检测到批次内的数据混合，此回调将使用以下消息警告用户：

In [ ]:
Your model is mixing data across the batch dimension.
This can lead to wrong gradient updates in the optimizer.
Check the operations that reshape and permute tensor dimensions in your model.

也可以使用适用于任何 PyTorch 模块的非回调版本 BatchGradientVerification：

In [ ]:
from pl_bolts.utils import BatchGradientVerification

model = YourPyTorchModel()
verification = BatchGradientVerification(model)
valid = verification.check(input_array=torch.rand(2, 3, 4), sample_idx=1)

在这个例子中，我们通过检查第二个样本的梯度，在批量大小 2 上运行测试。

## 自我监督的回调

自监督学习模型的有用回调

### BYOLMA权重更新

来自 Bootstrap Your Own Latent (BYOL) 的指数移动平均权重更新规则。

In [ ]:
# 模型必须有 2 个属性
model = Model()
model.online_network = ...
model.target_network = ...

trainer = Trainer(callbacks=[BYOLMAWeightUpdate()])

### SSL在线评估器

附加 MLP 以对给定模型进行微调。 回调有自己的迷你内部循环。

In [ ]:
# 您的模型必须有 2 个属性
model = Model()
model.z_dim = ... # the representation dim
model.num_classes = ... # the num of classes in the model

online_eval = SSLOnlineEvaluator(
    z_dim=model.z_dim,
    num_classes=model.num_classes,
    dataset='imagenet'
)

## 可变回调

对 GAN、可变自动编码器或任何具有潜在空间的东西有用的回调。

潜层纬度插值器

插值潜层纬度。

示例输出：

![](https://tva1.sinaimg.cn/large/008i3skNly1gu27oqrhqvj61600m0gov02.jpg)

In [ ]:
from pl_bolts.callbacks import LatentDimInterpolator

Trainer(callbacks=[LatentDimInterpolator()])

## 视觉回调

视觉模型的有用回调

### 混淆Logit

显示输入必须如何更改才能将预测从一个 logit 移动到另一个 logit

示例输出：

![](https://tva1.sinaimg.cn/large/008i3skNly1gu27qnaqs3j616m0fatat02.jpg)

In [ ]:
from pl_bolts.callbacks.vision import ConfusedLogitCallback
trainer = Trainer(callbacks=[ConfusedLogitCallback()])

> 每当调用时，此模型将在 LightningModule 中查找 self.last_batch 和 self.last_logits。

### Tensorboard 图像生成器

从生成模型生成图像并绘制到张量板

生成图像和日志到张量板。 您的模型必须实现生成的前向函数

In [ ]:
# model must have img_dim arg
model.img_dim = (1, 28, 28)

# model forward must work for sampling
z = torch.rand(batch_size, latent_dim)
img_samples = your_model(z)

例子：

In [ ]:
from pl_bolts.callbacks import TensorboardGenerativeModelImageSampler

trainer = Trainer(callbacks=[TensorboardGenerativeModelImageSampler()])

# 数据模块

在 PyTorch 中，处理数据具有以下主要元素。
* 下载、保存和准备数据集。
* 拆分为训练、验证和测试。
* 对于每个分割，应用不同的变换

DataModule 将这些操作组合到一个可重复的 DataModule 中，该 DataModule 可以共享以保证：
* 一致的数据预处理（下载、拆分等...）
* 完全相同的分裂
* 完全相同的变换

In [ ]:
from pl_bolts.datamodules import ImagenetDataModule

dm = ImagenetDataModule(data_dir=PATH)

# 标准的 PyTorch！
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

Trainer().fit(
    model,
    train_loader,
    val_loader
)

但是当与 PyTorch LightningModules（所有 Bolts 模型）搭配使用时，您可以使用相同的拆分、转换等插入和播放完整的数据集定义……

In [ ]:
imagenet = ImagenetDataModule(PATH)
model = VAE(datamodule=imagenet)
model = ImageGPT(datamodule=imagenet)
model = GAN(datamodule=imagenet)

我们甚至有预先构建的模块来弥合 Numpy、Sklearn 和 PyTorch 之间的差距

In [ ]:
from sklearn.datasets import load_diabetes
from pl_bolts.datamodules import SklearnDataModule

X, y = load_diabetes(return_X_y=True)
datamodule = SklearnDataModule(X, y)

model = LitModel(datamodule)

## 回归英雄

如果您的工作或研究不需要“锤子”，我们提供经典 ML 模型的实现，这些模型受益于 Lightning 的多 GPU 和 TPU 支持。

因此，现在您可以可扩展地运行大量工作负载，而无需进行任何工程设计。 例如，在这里我们可以在 Imagenet 上运行逻辑回归（每个 epoch 大约需要 3 分钟）！

In [ ]:
from pl_bolts.models.regression import LogisticRegression

imagenet = ImagenetDataModule(PATH)

# 224 x 224 x 3
pixels_per_image = 150528
model = LogisticRegression(input_dim=pixels_per_image, num_classes=1000)
model.prepare_data = imagenet.prepare_data

trainer = Trainer(gpus=2)
trainer.fit(
    model,
    imagenet.train_dataloader(batch_size=256),
    imagenet.val_dataloader(batch_size=256)
)

## 线性回归

这是线性回归的示例

In [ ]:
import pytorch_lightning as pl
from pl_bolts.datamodules import SklearnDataModule
from sklearn.datasets import load_diabetes

# link the numpy dataset to PyTorch
X, y = load_diabetes(return_X_y=True)
loaders = SklearnDataModule(X, y)

# training runs training batches while validating against a validation set
model = LinearRegression()
trainer = pl.Trainer(num_gpus=8)
trainer.fit(model, train_dataloader=loaders.train_dataloader(), val_dataloaders=loaders.val_dataloader())

完成后，您可以根据需要运行测试集。

In [ ]:
trainer.test(test_dataloaders=loaders.test_dataloader())

但更重要的是，您可以扩展到多个 GPU、TPU 甚至 CPU

In [ ]:
# 8 GPUs
trainer = pl.Trainer(num_gpus=8)

# 8 TPU cores
trainer = pl.Trainer(tpu_cores=8)

# 32 GPUs
trainer = pl.Trainer(num_gpus=8, num_nodes=4)

# 128 CPUs
trainer = pl.Trainer(num_processes=128)

## 逻辑回归

这是逻辑回归的一个例子

In [ ]:
from sklearn.datasets import load_iris
from pl_bolts.models.regression import LogisticRegression
from pl_bolts.datamodules import SklearnDataModule
import pytorch_lightning as pl

# use any numpy or sklearn dataset
X, y = load_iris(return_X_y=True)
dm = SklearnDataModule(X, y, batch_size=12)

# build model
model = LogisticRegression(input_dim=4, num_classes=3)

# fit
trainer = pl.Trainer(tpu_cores=8, precision=16)
trainer.fit(model, train_dataloader=dm.train_dataloader(), val_dataloaders=dm.val_dataloader())

trainer.test(test_dataloaders=dm.test_dataloader())

除了第一个（批次）之外，任何输入都将在所有维度上展平。 这意味着图像、声音等……开箱即用。

In [ ]:
# create dataset
dm = MNISTDataModule(num_workers=0, data_dir=tmpdir)

model = LogisticRegression(input_dim=28 * 28, num_classes=10, learning_rate=0.001)
model.prepare_data = dm.prepare_data
model.train_dataloader = dm.train_dataloader
model.val_dataloader = dm.val_dataloader
model.test_dataloader = dm.test_dataloader

trainer = pl.Trainer(max_epochs=2)
trainer.fit(model)
trainer.test(model)
# {test_acc: 0.92}

但更重要的是，您可以扩展到多个 GPU、TPU 甚至 CPU

In [ ]:
# 8 GPUs
trainer = pl.Trainer(num_gpus=8)

# 8 TPUs
trainer = pl.Trainer(tpu_cores=8)

# 32 GPUs
trainer = pl.Trainer(num_gpus=8, num_nodes=4)

# 128 CPUs
trainer = pl.Trainer(num_processes=128)

# 常规 PyTorch

Bolt 中的所有内容也适用于常规 PyTorch，因为它们都只是 nn.Modules！

但是，如果您使用 Lightning 进行训练，则无需处理工程代码 :)

# 命令行支持

也可以从命令行训练任何 bolt 模块
```bash
cd pl_bolts/models/autoencoders/basic_vae
python basic_vae_pl_module.py
```

每个脚本都接受闪电训练器和模型的 Argparse 参数
```bash
python basic_vae_pl_module.py --latent_dim 32 --batch_size 32 --gpus 4 --max_epochs 12
```

# 数据模块

DataModules（在 PyTorch Lightning 0.9.0 中引入）将数据与模型分离。 DataModule 只是训练数据加载器、验证数据加载器和测试数据加载器的集合。 此外，它还指定了如何：
* 下载/准备数据。
* 训练/验证/测试拆分。
* 转变
然后你可以像这样使用它：

例子：

In [ ]:
dm = MNISTDataModule('path/to/data')
model = LitModel()

trainer = Trainer()
trainer.fit(model, datamodule=dm)

或者通过普通的 PyTorch 手动使用它

例子：

In [ ]:
dm = MNISTDataModule('path/to/data')
for batch in dm.train_dataloader():
    ...
for batch in dm.val_dataloader():
    ...
for batch in dm.test_dataloader():
    ...

请访问 PyTorch Lightning 文档以获取有关 DataModules 的更多详细信息

## SklearnDataModule

将 sklearn 或 numpy 数据集映射到 PyTorch Dataloaders 的实用程序，具有自动数据拆分和 GPU/TPU 支持。

In [6]:
from sklearn.datasets import load_diabetes
from pl_bolts.datamodules import SklearnDataModule

X, y = load_diabetes(return_X_y=True)
loaders = SklearnDataModule(X, y, batch_size = 32)

train_loader = loaders.train_dataloader()
val_loader = loaders.val_dataloader()
test_loader = loaders.test_dataloader()

time: 19.5 ms (started: 2021-09-02 13:32:12 +08:00)


或构建您自己的火炬数据集

In [7]:
from sklearn.datasets import load_diabetes
from pl_bolts.datamodules import SklearnDataset

X, y = load_diabetes(return_X_y=True)
dataset = SklearnDataset(X, y)
loader = DataLoader(dataset)

time: 15.6 ms (started: 2021-09-02 13:32:57 +08:00)


## Sklearn 数据集类

将 sklearn 或 numpy 数据集转换为 PyTorch 数据集。

In [9]:
from sklearn.datasets import load_diabetes
from pl_bolts.datamodules import SklearnDataset

X, y = load_diabetes(return_X_y=True)
dataset = SklearnDataset(X, y)
len(dataset)

442

time: 17.8 ms (started: 2021-09-02 13:35:00 +08:00)


## Sklearn DataModule 类

为 Numpy 数据集自动生成训练、验证和测试拆分。 为方便起见，它们被设置为数据加载器。 或者，您可以传入自己的验证和测试拆分。

为 Numpy 数据集自动生成训练、验证和测试拆分。 为方便起见，它们被设置为数据加载器。 或者，您可以传入自己的验证和测试拆分。

例子

In [10]:
from sklearn.datasets import load_diabetes
from pl_bolts.datamodules import SklearnDataModule

X, y = load_diabetes(return_X_y=True)
loaders = SklearnDataModule(X, y, batch_size=32)

# train set
train_loader = loaders.train_dataloader()
len(train_loader.dataset)

310

time: 20 ms (started: 2021-09-02 13:38:21 +08:00)


In [11]:
len(train_loader)

10

time: 3.47 ms (started: 2021-09-02 13:38:24 +08:00)


In [12]:
# validation set
val_loader = loaders.val_dataloader()
len(val_loader.dataset)

88

time: 3.8 ms (started: 2021-09-02 13:38:36 +08:00)


In [13]:
len(val_loader)

3

time: 3.25 ms (started: 2021-09-02 13:38:39 +08:00)


In [14]:
# test set
test_loader = loaders.test_dataloader()
len(test_loader.dataset)

44

time: 4.17 ms (started: 2021-09-02 13:38:44 +08:00)


In [15]:
len(test_loader)

2

time: 3.41 ms (started: 2021-09-02 13:38:46 +08:00)


## 视觉数据模块

以下是用于计算机视觉的预构建数据模块。

**监督学习**

这些是标准视觉数据集，其中包含在 DataLoaders 中预先生成的训练、测试、val 分割以及标准变换（和归一化）值

### BinaryMNIST

In [ ]:
from pl_bolts.datamodules import BinaryMNISTDataModule

dm = BinaryMNISTDataModule('.')
model = LitModel()

Trainer().fit(model, datamodule=dm)

### CityScapes

变换：

In [ ]:
transforms = transform_lib.Compose([
    transform_lib.ToTensor(),
    transform_lib.Normalize(
        mean=[0.28689554, 0.32513303, 0.28389177],
        std=[0.18696375, 0.19017339, 0.18720214]
    )
])

例子：

In [ ]:
from pl_bolts.datamodules import CityscapesDataModule

dm = CityscapesDataModule(PATH)
model = LitModel()

Trainer().fit(model, datamodule=dm)

或者您可以设置自己的转换

例子：

In [ ]:
dm.train_transforms = ...
dm.test_transforms = ...
dm.val_transforms  = ...
dm.target_transforms = ...

### CIFAR-10

标准 CIFAR10、训练、验证、测试拆分和转换

变换：

In [ ]:
mnist_transforms = transform_lib.Compose([
    transform_lib.ToTensor(),
    transforms.Normalize(
        mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
        std=[x / 255.0 for x in [63.0, 62.1, 66.7]]
    )
])

例子：

In [ ]:
from pl_bolts.datamodules import CIFAR10DataModule

dm = CIFAR10DataModule(PATH)
model = LitModel()

Trainer().fit(model, datamodule=dm)

或者您可以设置自己的转换

例子：

In [ ]:
dm.train_transforms = ...
dm.test_transforms = ...
dm.val_transforms  = ...

### FashionMNIST

In [ ]:
from pl_bolts.datamodules import FashionMNISTDataModule

dm = FashionMNISTDataModule('.')
model = LitModel()

Trainer().fit(model, datamodule=dm)

### Imagenet

In [ ]:
from pl_bolts.datamodules import ImagenetDataModule

dm = ImagenetDataModule(IMAGENET_PATH)
model = LitModel()

Trainer().fit(model, datamodule=dm)

### MNIST

In [ ]:
from pl_bolts.datamodules import MNISTDataModule

dm = MNISTDataModule('.')
model = LitModel()

Trainer().fit(model, datamodule=dm)

**半监督学习**

以下数据集支持未标记训练和半监督学习，其中仅标记了几个示例。

### Imagenet (ssl)

### STL-10

In [ ]:
from pl_bolts.datamodules import STL10DataModule

dm = STL10DataModule(PATH)
model = LitModel()

Trainer().fit(model, datamodule=dm)

# 数据集

收集有用的数据集

使用这些数据集进行调试

## DummyDataset

In [16]:
from pl_bolts.datasets import DummyDataset
from torch.utils.data import DataLoader
# mnist dims
ds = DummyDataset((1, 28, 28), (1, ))
dl = DataLoader(ds, batch_size=7)
# get first batch
batch = next(iter(dl))
x, y = batch
x.size()

torch.Size([7, 1, 28, 28])

time: 6.35 ms (started: 2021-09-02 15:07:05 +08:00)


In [17]:
y.size()

torch.Size([7, 1])

time: 3.73 ms (started: 2021-09-02 15:07:07 +08:00)


## DummyDetectionDataset

生成用于检测的虚拟数据集

例子

In [18]:
from pl_bolts.datasets import DummyDetectionDataset
from torch.utils.data import DataLoader
ds = DummyDetectionDataset()
dl = DataLoader(ds, batch_size=7)

time: 1.53 ms (started: 2021-09-02 15:07:53 +08:00)


## RandomDataset

In [19]:
from pl_bolts.datasets import RandomDataset
from torch.utils.data import DataLoader
ds = RandomDataset(10)
dl = DataLoader(ds, batch_size=7)

time: 2.06 ms (started: 2021-09-02 15:08:39 +08:00)


## RandomDictDataset

生成具有 dict 结构的虚拟数据集

例子

In [20]:
from pl_bolts.datasets import RandomDictDataset
from torch.utils.data import DataLoader
ds = RandomDictDataset(10)
dl = DataLoader(ds, batch_size=7)

time: 1.85 ms (started: 2021-09-02 15:09:19 +08:00)


## RandomDictStringDataset

生成带有字符串的虚拟数据集

例子

In [21]:
from pl_bolts.datasets import RandomDictStringDataset
from torch.utils.data import DataLoader
ds = RandomDictStringDataset(10)
dl = DataLoader(ds, batch_size=7)

time: 1.53 ms (started: 2021-09-02 15:10:05 +08:00)


# DataLoaders

## AsynchronousLoader

此数据加载器的行为与标准 pytorch 数据加载器相同，但会通过训练将数据异步传输到 GPU。 您还可以使用它来包装现有的数据加载器。

例子：

In [ ]:
dataloader = AsynchronousLoader(DataLoader(ds, batch_size=16), device=device)

for b in dataloader:
    ...

使用 DataLoader 从 CPU 内存异步加载到设备内存的类。

请注意，这仅适用于单 GPU 训练，multiGPU 使用 PyTorch 的 DataParallel 或 DistributedDataParallel，后者使用自己的代码跨 GPU 传输数据。 这可能会破坏 DataParallel 或 DistributedDataParallel 或使事情变慢。

# 损失

这个包列出了跨研究领域的常见损失（这是一项正在进行的工作。如果你有任何损失想要贡献，请提交 PR！）

## 物体检测

这些是对象检测中使用的常见损失。

### GIoU Loss

In [22]:
import torch
from pl_bolts.losses.object_detection import giou_loss

preds = torch.tensor([[100, 100, 200, 200]])
target = torch.tensor([[150, 150, 250, 250]])
giou_loss(preds, target)

tensor([[1.0794]])

time: 43.2 ms (started: 2021-09-02 15:18:43 +08:00)


### IoU Loss

In [23]:
import torch
from pl_bolts.losses.object_detection import iou_loss
preds = torch.tensor([[100, 100, 200, 200]])
target = torch.tensor([[150, 150, 250, 250]])
iou_loss(preds, target)

tensor([[0.8571]])

time: 11.7 ms (started: 2021-09-02 15:19:25 +08:00)


## 强化学习

这些是 RL 中使用的常见损失。

### DQN Loss

使用重播缓冲区中的小批量计算 mse 损失

### Double DQN Loss

使用重播缓冲区中的小批量计算 mse 损失。 这通过使用双 dqn 对原始 DQN 损失进行了改进。 这通过使用训练网络的动作从目标网络中选择值来显示。 这段代码被大量注释，以便清楚地解释这个过程

### Per DQN Loss

使用来自 PER 缓冲区的批次的优先权重计算 mse 损失

# 模型

## 如何使用模型

模型旨在“固定”到您的研究或生产案例上。

Bolts旨在以下列方式使用

### 预测您的数据

大多数Bolts都有在各种数据集或算法上训练的预训练权重。 当您没有足够的数据、时间或金钱来进行自己的培训时，这很有用。

例如，您可以使用预训练的 VAE 为图像数据集生成特征。

In [2]:
from pl_bolts.models.self_supervised import SimCLR

weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
encoder = simclr.encoder
encoder.eval()

/data/miniconda3/envs/torch19/lib/python3.9/site-packages/pytorch_lightning/core/saving.py:209: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['non_linear_evaluator.block_forward.2.weight', 'non_linear_evaluator.block_forward.2.bias']
  rank_zero_warn(


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
for (x, y) in own_data:
    features = encoder(x)

Bolts的优点是每个系统都可以以有趣的方式分解和使用。 例如，这个 resnet50 在 Imagenet 上使用自监督学习（无标签）进行训练，因此可能比使用标签训练的相同 resnet50 表现更好

In [6]:
# 无标签训练
from pl_bolts.models.self_supervised import SimCLR

weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
resnet50_unsupervised = simclr.encoder.eval()

# 用标签训练
from torchvision.models import resnet50
resnet50_supervised = resnet50(pretrained=True)

In [ ]:
# 也许在没有标签的情况下训练的特征对于分类或其他任务要好得多
x = image_sample()
unsup_feats = resnet50_unsupervised(x)
sup_feats = resnet50_supervised(x)

# 哪个会更好？

Bolt 通常在多个数据集上进行训练。

In [8]:
from pl_bolts.models.self_supervised import SimCLR

# imagenet weights
weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)

simclr.freeze()

## 对数据进行微调

如果您有一点数据并且可以支付一点培训费用，那么对自己的数据进行微调通常会更好。

要进行微调，您有两个选项解冻微调或稍后解冻。

### 解冻微调

在这种方法中，我们加载预训练模型并从头开始解冻

In [ ]:
from pl_bolts.models.self_supervised import SimCLR

weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
resnet50 = simclr.encoder
# don't call .freeze()

In [ ]:
classifier = LogisticRegression(...)

for (x, y) in own_data:
    feats = resnet50(x)
    y_hat = classifier(feats)
    ...

或者作为 LightningModule

In [ ]:
class FineTuner(pl.LightningModule):

    def __init__(self, encoder):
        self.encoder = encoder
        self.classifier = LogisticRegression(...)

    def training_step(self, batch, batch_idx):
        (x, y) = batch
        feats = self.encoder(x)
        y_hat = self.classifier(feats)
        loss = cross_entropy_with_logits(y_hat, y)
        return loss

trainer = Trainer(gpus=2)
model = FineTuner(resnet50)
trainer.fit(model)

有时这很有效，但更多时候最好让编码器冻结一段时间

### 冻结然后解冻

最常用的方法是先冻结然后解冻

In [ ]:
# freeze!
from pl_bolts.models.self_supervised import SimCLR

weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
resnet50 = simclr.encoder
resnet50.eval()

In [ ]:
classifier = LogisticRegression(...)

for epoch in epochs:
    for (x, y) in own_data:
        feats = resnet50(x)
        y_hat = classifier(feats)
        loss = cross_entropy_with_logits(y_hat, y)

    # unfreeze after 10 epochs
    if epoch == 10:
        resnet50.unfreeze()

或者在 Lightning 中作为回调，这样你就不会污染你的研究代码。

In [ ]:
class UnFreezeCallback(Callback):

    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch == 10.
            encoder.unfreeze()

trainer = Trainer(gpus=2, callbacks=[UnFreezeCallback()])
model = FineTuner(resnet50)
trainer.fit(model)

除非你仍然需要将它混合到你的研究代码中。

In [ ]:
class FineTuner(pl.LightningModule):

    def __init__(self, encoder):
        self.encoder = encoder
        self.classifier = LogisticRegression(...)

    def training_step(self, batch, batch_idx):

        # option 1 - (not recommended because it's messy)
        if self.trainer.current_epoch == 10:
            self.encoder.unfreeze()

        (x, y) = batch
        feats = self.encoder(x)
        y_hat = self.classifier(feats)
        loss = cross_entropy_with_logits(y_hat, y)
        return loss

    def on_epoch_end(self, trainer, pl_module):
        # a hook is cleaner (but a callback is much better)
        if self.trainer.current_epoch == 10:
            self.encoder.unfreeze()

### 超参数搜索

为了使微调工作良好，您应该尝试模型超参数的许多版本。 否则，您不太可能从数据中获得最大价值。

In [ ]:
from pl_bolts.models.autoencoders import VAE

learning_rates = [0.01, 0.001, 0.0001]
hidden_dim = [128, 256, 512]

for lr in learning_rates:
    for hd in hidden_dim:
        vae = VAE(input_height=32, hidden_dim=hd, learning_rate=lr)
        trainer = Trainer()
        trainer.fit(vae)

## 从头开始训练

如果您确实有足够的数据和计算资源，那么您可以尝试从头开始训练。

In [ ]:
# get data
train_data = DataLoader(YourDataset)
val_data = DataLoader(YourDataset)

# use any bolts model without pretraining
model = VAE()

# fit!
trainer = Trainer(gpus=2)
trainer.fit(model, train_dataloader=train_data, val_dataloaders=val_data)

## 研究用

bolts 与所有其他库的区别在于，bolts 是由 AI 研究人员构建和使用的。 这意味着每个螺栓都是模块化的，因此可以轻松扩展或与代码库其余部分的任意部分混合。

### 扩展工作

也许一个研究项目需要修改已知方法的一部分。 在这种情况下，您最好只更改已知性能良好的系统部分。 否则，您可能无法正确执行工作。

示例 1：更改 VAE 的先验或近似后验

In [ ]:
from pl_bolts.models.autoencoders import VAE

class MyVAEFlavor(VAE):

    def init_prior(self, z_mu, z_std):
        P = MyPriorDistribution

        # default is standard normal
        # P = distributions.normal.Normal(loc=torch.zeros_like(z_mu), scale=torch.ones_like(z_std))
        return P

    def init_posterior(self, z_mu, z_std):
        Q = MyPosteriorDistribution
        # default is normal(z_mu, z_sigma)
        # Q = distributions.normal.Normal(loc=z_mu, scale=z_std)
        return Q

当然，用闪电训练它。

In [ ]:
model = MyVAEFlavor()
trainer = Trainer()
trainer.fit(model)

只需几行代码，您就可以更改 VAE 的一些基本内容……这意味着您可以更快地迭代想法，因为知道 bolt 实现和训练循环是正确且经过测试的。

如果您的模型不适用于新的 P、Q，那么您可以更快地放弃该研究想法，而不是试图确定您的 VAE 实施是否正确，或者您的训练循环是否正确。

示例 2：更改 GAN 的生成器步骤

In [ ]:
from pl_bolts.models.gans import GAN

class FancyGAN(GAN):

    def generator_step(self, x):
        # sample noise
        z = torch.randn(x.shape[0], self.hparams.latent_dim)
        z = z.type_as(x)

        # generate images
        self.generated_imgs = self(z)

        # ground truth result (ie: all real)
        real = torch.ones(x.size(0), 1)
        real = real.type_as(x)
        g_loss = self.generator_loss(real)

        tqdm_dict = {'g_loss': g_loss}
        output = OrderedDict({
            'loss': g_loss,
            'progress_bar': tqdm_dict,
            'log': tqdm_dict
        })
        return output

示例 3：在对比性自监督学习方法中改变计算损失的方式

In [ ]:
from pl_bolts.models.self_supervised import AMDIM

class MyDIM(AMDIM):

    def validation_step(self, batch, batch_nb):
        [img_1, img_2], labels = batch

        # generate features
        r1_x1, r5_x1, r7_x1, r1_x2, r5_x2, r7_x2 = self.forward(img_1, img_2)

        # Contrastive task
        loss, lgt_reg = self.contrastive_task((r1_x1, r5_x1, r7_x1), (r1_x2, r5_x2, r7_x2))
        unsupervised_loss = loss.sum() + lgt_reg

        result = {
            'val_nce': unsupervised_loss
        }
        return result

## 导入零件

所有螺栓都是模块化的。 这意味着您还可以任意混合和匹配来自不同方法的基本块。

示例 1：将 GAN 的 VAE 编码器用作生成器

In [ ]:
from pl_bolts.models.gans import GAN
from pl_bolts.models.autoencoders.basic_vae import Encoder

class FancyGAN(GAN):

    def init_generator(self, img_dim):
        generator = Encoder(...)
        return generator

trainer = Trainer(...)
trainer.fit(FancyGAN())

例2：在CPC中使用AMDIM的对比任务

In [ ]:
from pl_bolts.models.self_supervised import AMDIM, CPC_v2

default_amdim_task = AMDIM().contrastive_task
model = CPC_v2(contrastive_task=default_amdim_task, encoder='cpc_default')
# you might need to modify the cpc encoder depending on what you use

## 组成新想法

您可能也有兴趣创建全新的方法来混合和匹配各种不同的部分

In [ ]:
# 此模型仅用于说明目的，没有研究意义，但旨在表明您可以随心所欲地发挥创造力和表现力。
class MyNewContrastiveApproach(pl.LightningModule):

    def __init__(self):
        suoer().__init_()

        self.gan = GAN()
        self.vae = VAE()
        self.amdim = AMDIM()
        self.cpc = CPC_v2

    def training_step(self, batch, batch_idx):
        (x, y) = batch

        feat_a = self.gan.generator(x)
        feat_b = self.vae.encoder(x)

        unsup_loss = self.amdim(feat_a) + self.cpc(feat_b)

        vae_loss = self.vae._step(batch)
        gan_loss = self.gan.generator_loss(x)

        return unsup_loss + vae_loss + gan_loss

## 经典机器学习模型

该模块在 PyTorch Lightning 中实现了经典的机器学习模型，包括线性回归和逻辑回归。 与其他实现这些模型的库不同，这里我们使用 PyTorch 来启用多 GPU、多 TPU 和半精度训练。

### 线性回归

线性回归拟合了实值目标变量 y 和一个或多个特征 X 之间的线性模型。我们估计了最小化预测和真实目标值之间的均方误差的回归系数。

我们将线性回归模型制定为单层神经网络。 默认情况下，我们在输出层中只包含一个神经元，尽管您可以自己指定 output_dim。

通过指定正则化强度（默认为 0）添加 L1 或 L2 正则化，或两者。

In [ ]:
from pl_bolts.models.regression import LinearRegression
import pytorch_lightning as pl
from pl_bolts.datamodules import SklearnDataModule
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
loaders = SklearnDataModule(X, y)

model = LinearRegression(input_dim=13)
trainer = pl.Trainer()
trainer.fit(model, train_dataloader=loaders.train_dataloader(), val_dataloaders=loaders.val_dataloader())
trainer.test(test_dataloaders=loaders.test_dataloader())

### 逻辑回归

逻辑回归是用于分类的线性模型，即当我们有一个分类目标变量时。 此实现支持二元和多类分类。

在二元情况下，我们将逻辑回归模型表述为一层神经网络，输出层有一个神经元，并有一个 sigmoid 激活函数。 在多类情况下，我们使用单层神经网络，但现在输出中有 k 个神经元，其中 k 是类的数量。 这也称为多项逻辑回归。

通过指定正则化强度（默认为 0）添加 L1 或 L2 正则化，或两者。

In [ ]:
from sklearn.datasets import load_iris
from pl_bolts.models.regression import LogisticRegression
from pl_bolts.datamodules import SklearnDataModule
import pytorch_lightning as pl

# use any numpy or sklearn dataset
X, y = load_iris(return_X_y=True)
dm = SklearnDataModule(X, y)

# build model
model = LogisticRegression(input_dim=4, num_classes=3)

# fit
trainer = pl.Trainer(tpu_cores=8, precision=16)
trainer.fit(model, train_dataloader=dm.train_dataloader(), val_dataloaders=dm.val_dataloader())

trainer.test(test_dataloaders=dm.test_dataloader(batch_size=12))

除了第一个（批次）之外，任何输入都将在所有维度上展平。 这意味着图像、声音等……开箱即用。

In [ ]:
# create dataset
dm = MNISTDataModule(num_workers=0, data_dir=tmpdir)

model = LogisticRegression(input_dim=28 * 28, num_classes=10, learning_rate=0.001)
model.prepare_data = dm.prepare_data
model.train_dataloader = dm.train_dataloader
model.val_dataloader = dm.val_dataloader
model.test_dataloader = dm.test_dataloader

trainer = pl.Trainer(max_epochs=2)
trainer.fit(model)
trainer.test(model)
# {test_acc: 0.92}

# 视觉模型

## 自编码器

本节包含自动编码器和变分自动编码器。

### 基本AE

这是最简单的自动编码器。 你可以像这样使用它

In [ ]:
from pl_bolts.models.autoencoders import AE

model = AE()
trainer = Trainer()
trainer.fit(model)

您可以覆盖此 AE 的任何部分来构建您自己的变体。

In [ ]:
from pl_bolts.models.autoencoders import AE

class MyAEFlavor(AE):

    def init_encoder(self, hidden_dim, latent_dim, input_width, input_height):
        encoder = YourSuperFancyEncoder(...)
        return encoder

您可以使用螺栓中存在的预训练模型。

CIFAR-10 预训练模型：

In [ ]:
from pl_bolts.models.autoencoders import AE

ae = AE(input_height=32)
print(AE.pretrained_weights_available())
ae = ae.from_pretrained('cifar10-resnet18')

ae.freeze()

重建：

输入和生成的图像都是标准化版本，因为训练是用这些图像完成的。

In [ ]:
# not pretrained
ae = AE()

# pretrained on cifar10
ae = AE(input_height=32).from_pretrained('cifar10-resnet18')

### 变分自编码器

**基础VAE**

像这样使用 VAE。

In [ ]:
from pl_bolts.models.autoencoders import VAE

model = VAE()
trainer = Trainer()
trainer.fit(model)

您可以覆盖此 VAE 的任何部分以构建您自己的变体。

In [ ]:
from pl_bolts.models.autoencoders import VAE

class MyVAEFlavor(VAE):

    def get_posterior(self, mu, std):
        # do something other than the default
        # P = self.get_distribution(self.prior, loc=torch.zeros_like(mu), scale=torch.ones_like(std))

        return P

您可以使用螺栓中存在的预训练模型。

CIFAR-10 预训练模型：

In [ ]:
from pl_bolts.models.autoencoders import VAE

vae = VAE(input_height=32)
print(VAE.pretrained_weights_available())
vae = vae.from_pretrained('cifar10-resnet18')

vae.freeze()

STL-10 预训练模型：

In [ ]:
from pl_bolts.models.autoencoders import VAE

vae = VAE(input_height=96, first_conv=True)
print(VAE.pretrained_weights_available())
vae = vae.from_pretrained('cifar10-resnet18')

vae.freeze()

具有高斯先验和近似后验的标准 VAE。

模型可在不同的数据集上进行预训练：

例子：

In [ ]:
# not pretrained
vae = VAE()

# pretrained on cifar10
vae = VAE(input_height=32).from_pretrained('cifar10-resnet18')

# pretrained on stl10
vae = VAE(input_height=32).from_pretrained('stl10-resnet18')

## 卷积架构

这个包列出了贡献的卷积架构。

### GPT-2

In [ ]:
from pl_bolts.models.vision import GPT2

seq_len = 17
batch_size = 32
vocab_size = 16
x = torch.randint(0, vocab_size, (seq_len, batch_size))
model = GPT2(embed_dim=32, heads=2, layers=2, num_positions=seq_len, vocab_size=vocab_size, num_classes=4)
results = model(x)

### Image GPT

In [ ]:
import pytorch_lightning as pl
from pl_bolts.models.vision import ImageGPT

dm = MNISTDataModule('.')
model = ImageGPT(dm)

pl.Trainer(gpu=4).fit(model)

作为脚本：
```bash
cd pl_bolts/models/vision/image_gpt
python igpt_module.py --learning_rate 1e-2 --batch_size 32 --gpus 4
```

### 像素CNN

In [3]:
from pl_bolts.models.vision import PixelCNN
import torch

model = PixelCNN(input_channels=3)
x = torch.rand(5, 3, 64, 64)
out = model(x)

out.shape

torch.Size([5, 3, 64, 64])

### UNet

### 语义分割

用于语义分割任务的模型模板。 该模型默认使用 UNet 架构。 覆盖此模型的任何部分以构建您自己的变体。

In [ ]:
from pl_bolts.models.vision import SemSegment
from pl_bolts.datamodules import KittiDataModule
import pytorch_lightning as pl

dm = KittiDataModule('path/to/kitt/dataset/', batch_size=4)
model = SemSegment(datamodule=dm)
trainer = pl.Trainer()
trainer.fit(model)

## GANs


生成对抗网络的集合

### 基本的 GAN

这是一个普通的 GAN。 该模型适用于任何数据集大小，但结果显示为 MNIST。 替换编码器、解码器或训练循环的任何部分以构建新方法，或者只是对数据进行微调。

In [ ]:
from pl_bolts.models.gans import GAN
...
gan = GAN()
trainer = Trainer()
trainer.fit(gan)

命令行：

In [ ]:
# mnist
python  basic_gan_module.py --gpus 1

# imagenet
python  basic_gan_module.py --gpus 1 --dataset 'imagenet2012'
--data_dir /path/to/imagenet/folder/ --meta_dir ~/path/to/meta/bin/folder
--batch_size 256 --learning_rate 0.0001

In [ ]:
z = torch.rand(batch_size, latent_dim)
gan = GAN.load_from_checkpoint(PATH)
img = gan(z)

### DCGAN

DCGAN 实现来自论文无监督表示学习与深度卷积生成对抗网络。 该实现基于 PyTorch 示例中的版本。

In [ ]:
from pl_bolts.models.gans import DCGAN

m = DCGAN()
Trainer(gpus=2).fit(m)

命令行：

In [ ]:
# mnist
python dcgan_module.py --gpus 1

# cifar10
python dcgan_module.py --gpus 1 --dataset cifar10 --image_channels 3

forward:

In [ ]:
noise = torch.rand(batch_size, latent_dim)
gan = GAN.load_from_checkpoint(PATH)
img = gan(noise)

## 强化学习

该模块是在 Lightning 中实现的常见 RL 方法的集合。
* mDQN
* Double DQN
* Dueling DQN
* Noisy DQN
* NStep DQN
* Prioritized Experience Replay DQN
* Reinforce
* Vanilla Policy Gradient

> RL 模型目前仅支持分布式后端=dp 的 CPU 和单 GPU 训练。 完整的 GPU 支持将在以后的更新中添加。

### DQN 模型

以下模型基于 DQN。 DQN 使用基于值的学习，它根据模型的当前学习值 (V) 或当前状态的状态动作值 (Q) 决定采取什么行动。 这些值被定义为代理状态或状态动作对的折扣总奖励。

深 Q 网络 (### DQN)

DQN 模型在 Playing Atari with Deep Reinforcement Learning 中引入。 论文作者：Volodymyr Mnih、Koray Kavukcuoglu、David Silver、Alex Graves、Ioannis Antonoglou、Daan Wierstra、Martin Riedmiller。

原始实施者：Donal Byrne

DeepMind 的研究人员在 Playing Atari with Deep Reinforcement Learning 中引入了 DQN。 这采用了表格 Q 学习的概念，并通过使用深度神经网络逼近 Q 函数将其扩展到更大的问题。

DQN 背后的目标是采用 Q 学习的简单控制方法并将其扩展以解决复杂的任务。 除此之外，该方法还需要稳定。 DQN 通过以下添加解决了这些问题。

近似 Q 函数

将 Q 值存储在表中理论上效果很好，但完全不可扩展。 相反，作者使用深度神经网络近似 Q 函数。 这允许 DQN 用于更复杂的任务

In [ ]:
from pl_bolts.models.rl import DQN
dqn = DQN("PongNoFrameskip-v4")
trainer = Trainer()
trainer.fit(dqn)

### 双DQN

双 DQN 模型在深度强化学习中引入双 Q 学习论文作者：Hado van Hasselt、Arthur Guez、David Silver

原始实施者：Donal Byrne

原始 DQN 在 Bellman 更新过程中往往会高估 Q 值，导致不稳定并且对训练有害。这是由于贝尔曼方程中的最大运算。

在我们的更新过程中，我们不断地采用我们的代理估计的最大值。如果我们可以相信这些估计，这似乎是合理的。然而，在训练的早期阶段，这些值的估计将偏离中心，并可能导致训练不稳定，直到我们的估计变得更加可靠

Double DQN 通过使用主要训练网络为下一个状态选择动作来修复这种高估，但使用来自更稳定目标网络的这些动作的值。所以我们仍然会采取贪婪的行动，但该值将不那么“乐观”，因为它是由目标网络选择的。

In [ ]:
from pl_bolts.models.rl import DoubleDQN
ddqn = DoubleDQN("PongNoFrameskip-v4")
trainer = Trainer()
trainer.fit(ddqn)

### Dueling DQN

决斗 DQN 的好处
* 能够有效地学习状态值函数。 在决斗网络中，每次 Q 更新也会更新价值流，而在 DQN 中，仅更新所选动作的值。 这提供了更好的值近似值
* 给定状态的总 Q 值之间的差异与 Q 的大小有关。 最佳动作和次佳动作之间的 Q 值差异可能非常小，而平均状态值可能要大得多 . 规模的差异会引入噪音，这可能会导致贪婪的策略切换这些动作的优先级。 状态价值和优势的单独估计器使 Duling DQN 对这种类型的场景具有鲁棒性

In [ ]:
from pl_bolts.models.rl import DuelingDQN
dueling_dqn = DuelingDQN("PongNoFrameskip-v4")
trainer = Trainer()
trainer.fit(dueling_dqn)

### Noisy DQN

嘈杂的 DQN 的好处
* 改进了探索功能。 我们不是仅仅执行完全随机的动作，而是在我们的策略中增加了减少的噪音和不确定性，允许在仍然利用其策略的同时进行探索。
* 该方法自动调整的事实意味着我们不必为 epsilon-greedy 调整超参数！

In [ ]:
from pl_bolts.models.rl import NoisyDQN
noisy_dqn = NoisyDQN("PongNoFrameskip-v4")
trainer = Trainer()
trainer.fit(noisy_dqn)

### N-Step DQN

N步优势
* 多步学习能够比典型的一步学习方法更快地学习。
* 请注意，此方法引入了一个新的超参数 n。 尽管 n=4 通常是一个很好的起点，并提供了全面的良好结果。

In [ ]:
from pl_bolts.models.rl import DQN
n_step_dqn = DQN("PongNoFrameskip-v4", n_steps=4)
trainer = Trainer()
trainer.fit(n_step_dqn)

### 优先体验重放 DQN

In [ ]:
from pl_bolts.models.rl import PERDQN
per_dqn = PERDQN("PongNoFrameskip-v4")
trainer = Trainer()
trainer.fit(per_dqn)

### 策略梯度模型

以下模型基于策略梯度。与之前展示的 Q 学习模型不同，基于策略的模型不会尝试学习状态或状态动作对的特定值。相反，它省去了中间人，直接学习策略分布。在策略梯度模型中，我们按照策略梯度建议的方向更新网络参数，以找到产生最高结果的策略。
政策梯度要点：
* 输出动作的分布而不是离散的 Q 值
* 直接优化策略，而不是通过优化 Q 值间接优化策略
* 动作的策略分布允许模型处理更复杂的动作空间，例如连续动作
* 策略分布引入了随机性，为模型提供了自然探索
* 策略分布提供了更稳定的更新，因为权重的变化只会稍微改变总分布，而不是基于状态 S 的 Q 值更改权重会改变具有相似状态的所有 Q 值。

策略梯度趋向于快速收敛，但是它们的样本效率不高，并且通常需要与环境进行更多的交互。

### REINFORCE

强化优点
* 简单明了
* 对于 Cartpole 等简单任务而言，计算效率比基于值的方法更高效。

In [ ]:
from pl_bolts.models.rl import Reinforce
reinforce = Reinforce("CartPole-v0")
trainer = Trainer()
trainer.fit(reinforce)

### Vanilla Policy Gradient

VPG 优点
* 添加基线减少了模型中的方差
* 由于熵奖励改进了探索

In [ ]:
from pl_bolts.models.rl import VanillaPolicyGradient
vpg = VanillaPolicyGradient("CartPole-v0")
trainer = Trainer()
trainer.fit(vpg)

## 自监督学习

该 bolts 模块包含所有自监督学习模型的集合。

自监督学习通过解决借口任务来提取输入的表示。 在这个包中，我们实现了许多当前最先进的自监督算法。

使用未标记的数据集训练自监督模型

### 提取图像特征

该模块中的模型是无监督训练的，因此可以捕获更好的图像表示（特征）。

在这个例子中，我们将加载一个 resnet 18，它使用 CPC 作为借口任务在 imagenet 上进行了预训练。

In [ ]:
from pl_bolts.models.self_supervised import SimCLR

# load resnet50 pretrained using SimCLR on imagenet
weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)

simclr_resnet50 = simclr.encoder
simclr_resnet50.eval()

这意味着您现在可以提取通过无监督学习进行预训练的图像表示。

例子：

In [ ]:
my_dataset = SomeDataset()
for batch in my_dataset:
    x, y = batch
    out = simclr_resnet50(x)

### 使用未标记的数据进行训练

当您拥有大量未标记的图像时，这些模型非常适合从头开始训练

In [ ]:
from pl_bolts.models.self_supervised import SimCLR
from pl_bolts.models.self_supervised.simclr import SimCLREvalDataTransform, SimCLRTrainDataTransform


train_dataset = MyDataset(transforms=SimCLRTrainDataTransform())
val_dataset = MyDataset(transforms=SimCLREvalDataTransform())

# simclr needs a lot of compute!
model = SimCLR()
trainer = Trainer(tpu_cores=128)
trainer.fit(
    model,
    DataLoader(train_dataset),
    DataLoader(val_dataset),
)

**研究**

混合和匹配任何部分或子类以创建您自己的新方法

In [ ]:
from pl_bolts.models.self_supervised import CPC_v2
from pl_bolts.losses.self_supervised_learning import FeatureMapContrastiveTask

amdim_task = FeatureMapContrastiveTask(comparisons='01, 11, 02', bidirectional=True)
model = CPC_v2(contrastive_task=amdim_task)

### 对比学习模型

对比自监督学习 (CSL) 是一种自监督学习方法，我们生成实例的表示，使得相似的实例彼此靠近而远离不同的实例。 这通常通过比较正、锚和负表示的三元组来完成。

在本节中，我们列出了流行的对比学习方法的 Lightning 实现。

#### AMDIM

In [4]:
from pl_bolts.models.self_supervised import AMDIM

model = AMDIM(encoder='resnet18')

训练:

In [ ]:
trainer = Trainer()
trainer.fit(model)

#### BYOL

In [ ]:
model = BYOL(num_classes=10)

dm = CIFAR10DataModule(num_workers=0)
dm.train_transforms = SimCLRTrainDataTransform(32)
dm.val_transforms = SimCLREvalDataTransform(32)

trainer = pl.Trainer()
trainer.fit(model, datamodule=dm)

训练：

In [ ]:
trainer = Trainer()
trainer.fit(model)

CLI命令：

In [ ]:
# cifar10
python byol_module.py --gpus 1

# imagenet
python byol_module.py
    --gpus 8
    --dataset imagenet2012
    --data_dir /path/to/imagenet/
    --meta_dir /path/to/folder/with/meta.bin/
    --batch_size 32

#### CPC (V2)

In [ ]:
import pytorch_lightning as pl
from pl_bolts.models.self_supervised import CPC_v2
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.models.self_supervised.cpc import (
    CPCTrainTransformsCIFAR10, CPCEvalTransformsCIFAR10)

# data
dm = CIFAR10DataModule(num_workers=0)
dm.train_transforms = CPCTrainTransformsCIFAR10()
dm.val_transforms = CPCEvalTransformsCIFAR10()

# model
model = CPC_v2()

# fit
trainer = pl.Trainer()
trainer.fit(model, datamodule=dm)

In [ ]:
python cpc_finetuner.py
    --ckpt_path path/to/checkpoint.ckpt
    --dataset cifar10
    --gpus 1

#### Moco (v2)

#### SimCLR

In [ ]:
import pytorch_lightning as pl
from pl_bolts.models.self_supervised import SimCLR
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.models.self_supervised.simclr.transforms import (
    SimCLREvalDataTransform, SimCLRTrainDataTransform)

# data
dm = CIFAR10DataModule(num_workers=0)
dm.train_transforms = SimCLRTrainDataTransform(32)
dm.val_transforms = SimCLREvalDataTransform(32)

# model
model = SimCLR(num_samples=dm.num_samples, batch_size=dm.batch_size, dataset='cifar10')

# fit
trainer = pl.Trainer()
trainer.fit(model, datamodule=dm)

CIFAR-10 预训练模型：

In [ ]:
from pl_bolts.models.self_supervised import SimCLR

weight_path = 'https://pl-bolts-weights.s3.us-east-2.amazonaws.com/simclr/bolts_simclr_imagenet/simclr_imagenet.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)

simclr.freeze()

重现：

In [ ]:
# pretrain
python simclr_module.py
    --gpus 8
    --dataset cifar10
    --batch_size 256
    -- num_workers 16
    --optimizer sgd
    --learning_rate 1.5
    --exclude_bn_bias
    --max_epochs 800
    --online_ft

# finetune
python simclr_finetuner.py
    --gpus 4
    --ckpt_path path/to/simclr/ckpt
    --dataset cifar10
    --batch_size 64
    --num_workers 8
    --learning_rate 0.3
    --num_epochs 100

#### SwAV

In [ ]:
import pytorch_lightning as pl
from pl_bolts.models.self_supervised import SwAV
from pl_bolts.datamodules import STL10DataModule
from pl_bolts.models.self_supervised.swav.transforms import (
    SwAVTrainDataTransform, SwAVEvalDataTransform
)
from pl_bolts.transforms.dataset_normalizations import stl10_normalization

# data
batch_size = 128
dm = STL10DataModule(data_dir='.', batch_size=batch_size)
dm.train_dataloader = dm.train_dataloader_mixed
dm.val_dataloader = dm.val_dataloader_mixed

dm.train_transforms = SwAVTrainDataTransform(
    normalize=stl10_normalization()
)

dm.val_transforms = SwAVEvalDataTransform(
    normalize=stl10_normalization()
)

# model
model = SwAV(
    gpus=1,
    num_samples=dm.num_unlabeled_samples,
    dataset='stl10',
    batch_size=batch_size
)

# fit
trainer = pl.Trainer(precision=16)
trainer.fit(model)

#### SimSiam

In [ ]:
model = SimSiam()

dm = CIFAR10DataModule(num_workers=0)
dm.train_transforms = SimCLRTrainDataTransform(32)
dm.val_transforms = SimCLREvalDataTransform(32)

trainer = pl.Trainer()
trainer.fit(model, datamodule=dm)

In [ ]:
trainer = Trainer()
trainer.fit(model)

# 学习率调度器

这个包列出了跨研究领域的常见学习率调度程序（这是一项正在进行的工作。如果你有任何想要贡献的学习率调度程序，请提交 PR！）

## 线性预热余弦退火学习率调度器

In [ ]:
layer = nn.Linear(10, 1)
optimizer = Adam(layer.parameters(), lr=0.02)
scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs=10, max_epochs=40)
#
# the default case
for epoch in range(40):
    # train(...)
    # validate(...)
    scheduler.step()
#
# passing epoch param case
for epoch in range(40):
    scheduler.step(epoch)
    # train(...)
    # validate(...)

# 自监督学习的变换

这些变换用于各种自监督学习方法。

## CPC转换

用于CPC变换  
可用的变换：
```
random_flip
img_jitter
col_jitter
rnd_gray
transforms.ToTensor()
normalize
Patchify(patch_size=patch_size, overlap_size=patch_size // 2)
```

例子：

In [ ]:
# in a regular dataset
CIFAR10(..., transforms=CPCTrainTransformsCIFAR10())

# in a DataModule
module = CIFAR10DataModule(PATH)
train_loader = module.train_dataloader(batch_size=32, transforms=CPCTrainTransformsCIFAR10())

### CIFAR-10 Eval (c)

包含的变换：
* random_flip
* transforms.ToTensor()
* normalize
* Patchify(patch_size=patch_size, overlap_size=overlap)

例子：

In [ ]:
# in a regular dataset
CIFAR10(..., transforms=CPCEvalTransformsCIFAR10())

# in a DataModule
module = CIFAR10DataModule(PATH)
train_loader = module.train_dataloader(batch_size=32, transforms=CPCEvalTransformsCIFAR10())

### Imagenet Train (c)

* random_flip
* transforms.ToTensor()
* normalize
* Patchify(patch_size=patch_size, overlap_size=patch_size // 2)

In [ ]:
# in a regular dataset
Imagenet(..., transforms=CPCTrainTransformsImageNet128())

# in a DataModule
module = ImagenetDataModule(PATH)
train_loader = module.train_dataloader(batch_size=32, transforms=CPCTrainTransformsImageNet128())

### Imagenet Eval (c)

* random_flip
* transforms.ToTensor()
* normalize
* Patchify(patch_size=patch_size, overlap_size=patch_size // 2)

In [ ]:
# in a regular dataset
Imagenet(..., transforms=CPCEvalTransformsImageNet128())

# in a DataModule
module = ImagenetDataModule(PATH)
train_loader = module.train_dataloader(batch_size=32, transforms=CPCEvalTransformsImageNet128())

### STL-10 火车 (c)

* random_flip
* img_jitter
* col_jitter
* rnd_gray
* transforms.ToTensor()
* normalize
* Patchify(patch_size=patch_size, overlap_size=patch_size // 2)

In [ ]:
# in a regular dataset
STL10(..., transforms=CPCTrainTransformsSTL10())

# in a DataModule
module = STL10DataModule(PATH)
train_loader = module.train_dataloader(batch_size=32, transforms=CPCTrainTransformsSTL10())

### STL-10 Eval (c)

* random_flip
* transforms.ToTensor()
* normalize
* Patchify(patch_size=patch_size, overlap_size=patch_size // 2)

In [ ]:
# in a regular dataset
STL10(..., transforms=CPCEvalTransformsSTL10())

# in a DataModule
module = STL10DataModule(PATH)
train_loader = module.train_dataloader(batch_size=32, transforms=CPCEvalTransformsSTL10())

## AMDIM 转换

用于 AMDIM 的变换

### CIFAR-10 Train (a)

### CIFAR-10 Eval (a)

### Imagenet Train (a)

### Imagenet Eval (a)

### STL-10 Train (a)

### STL-10 Eval (a)

## MOCO V2 变换

用于 MOCO V2 的变换

## SimCLR 变换

用于 SimCLR 的转换

# 自监督学习函数 

用于自监督学习的有用函数的集合

## Identity class
用于替换预训练模型中任意层的身份类
例子：

In [ ]:
from pl_bolts.utils import Identity

model = resnet18()
model.fc = Identity()

## SSL-ready resnets

Torchvision resnets去除了 fc 层，并且能够返回所有特征图，而不仅仅是最后一个。

例子：

In [ ]:
from pl_bolts.utils.self_supervised import torchvision_ssl_encoder

resnet = torchvision_ssl_encoder('resnet18', pretrained=False, return_all_feature_maps=True)
x = torch.rand(3, 3, 32, 32)

feat_maps = resnet(x)

## SSL backbone finetuner

使用具有 1024 个单元的单层 MLP 的标准评估协议对自监督学习主干进行微调

例子：

In [ ]:
from pl_bolts.utils.self_supervised import SSLFineTuner
from pl_bolts.models.self_supervised import CPC_v2
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.models.self_supervised.cpc.transforms import CPCEvalTransformsCIFAR10,
                                                            CPCTrainTransformsCIFAR10

# pretrained model
backbone = CPC_v2.load_from_checkpoint(PATH, strict=False)

# dataset + transforms
dm = CIFAR10DataModule(data_dir='.')
dm.train_transforms = CPCTrainTransformsCIFAR10()
dm.val_transforms = CPCEvalTransformsCIFAR10()

# finetuner
finetuner = SSLFineTuner(backbone, in_features=backbone.z_dim, num_classes=backbone.num_classes)

# train
trainer = pl.Trainer()
trainer.fit(finetuner, dm)

# test
trainer.test(datamodule=dm)

# 半监督学习

用于半监督学习的实用程序集合，其中部分数据已标记，另一部分未标记

## 平衡类

例子：

In [ ]:
from pl_bolts.utils.semi_supervised import balance_classes

确保每个批次的每个类都有相同数量的数据。 完美平衡

## 半标记批次

例子：

In [ ]:
from pl_bolts.utils.semi_supervised import generate_half_labeled_batches

给定一个标记数据集和一个未标记数据集，该函数生成一个联合对，其中一半批次被标记，另一半没有。

# 自我监督学习对比任务

本节实现了自监督学习中使用的流行对比学习任务。

## FeatureMapContrastiveTask

此任务比较特征图集。

一般来说，特征图比较借口任务使用特征的三元组。 以下是比较的抽象步骤。

生成同一图像的多个视图

In [ ]:
x1_view_1 = data_augmentation(x1)
x1_view_2 = data_augmentation(x1)

使用不同的示例生成额外的视图（通常在同一批次或候选池中）

In [ ]:
x2_view_1 = data_augmentation(x2)
x2_view_2 = data_augmentation(x2)

选择 3 个视图进行比较，这些是锚点、正面和负面特征

In [ ]:
anchor = x1_view_1
positive = x1_view_2
negative = x2_view_1

为每个视图生成特征图

In [ ]:
(a0, a1, a2) = encoder(anchor)
(p0, p1, p2) = encoder(positive)

对一组特征图进行比较

In [ ]:
phi = some_score_function()

# the '01' comparison
score = phi(a0, p1)

# and can be bidirectional
score = phi(p0, a1)

在实践中，对比任务创建了一个 BxB 矩阵，其中 B 是批量大小。 特征图集合 1 的对角线是锚点，特征图集合 2 的对角线是正数，集合 1 的非对角线是负数。

对每个传递的特征图元组执行锚点、正负对比较。

In [ ]:
# extract feature maps
pos_0, pos_1, pos_2 = encoder(x_pos)
anc_0, anc_1, anc_2 = encoder(x_anchor)

# compare only the 0th feature maps
task = FeatureMapContrastiveTask('00')
loss, regularizer = task((pos_0), (anc_0))

# compare (pos_0 to anc_1) and (pos_0, anc_2)
task = FeatureMapContrastiveTask('01, 02')
losses, regularizer = task((pos_0, pos_1, pos_2), (anc_0, anc_1, anc_2))
loss = losses.sum()

# compare (pos_1 vs a anc_random)
task = FeatureMapContrastiveTask('0r')
loss, regularizer = task((pos_0, pos_1, pos_2), (anc_0, anc_1, anc_2))

In [ ]:
# with bidirectional the comparisons are done both ways
task = FeatureMapContrastiveTask('01, 02')

# will compare the following:
# 01: (pos_0, anc_1), (anc_0, pos_1)
# 02: (pos_0, anc_2), (anc_0, pos_2)

Forward example:

In [ ]:
>>> import torch
>>> from pytorch_lightning import seed_everything
>>> seed_everything(0)
0
>>> a1 = torch.rand(3, 5, 2, 2)
>>> a2 = torch.rand(3, 5, 2, 2)
>>> b1 = torch.rand(3, 5, 2, 2)
>>> b2 = torch.rand(3, 5, 2, 2)
...
>>> task = FeatureMapContrastiveTask('01, 11')
...
>>> losses, regularizer = task((a1, a2), (b1, b2))
>>> losses
tensor([2.2351, 2.1902])
>>> regularizer
tensor(0.0324)